In [1]:
import requests
from bs4 import BeautifulSoup, element
import math
import re
import datetime

In [ ]:
def get_common_pickup(soup):
    gacha_list = []
    
    Flag =  False

    ID = 0
    rate = {'R' : 85.0, 'SR' : 12.0, 'SSR' : 3.0}
    time = 0
    sr_pickup_list = [] #2.4%
    ssr_pickup_list_0 = []
    ssr_pickup_list_1 = [] #1.2%

    for i in soup.select('.container'):
        tmp = i.select('p')
        if len(tmp) != 0 and i.select['p'][0].text == 'Current events and gachas:':
            gacha_list = i.select('.he_gacha')
            for j in gacha_list:
                time = j.select('.counter')[0]['data-count-to']
                if len(j.select('small')) != 0:
                    rate_tmp = i.select('small')[0].text
                    rate_tmp = rate_tmp.split('/')
                    rate_tmp[0] = rate_tmp[0].split(':')[1]
                    rate_tmp = list(map(lambda x : x.strip(), rate_tmp))

                    rate['R'] = float(rate_tmp[0].split(' ')[1][:-1])
                    rate['SR'] = float(rate_tmp[1].split(' ')[1][:-1])
                    rate['SSR'] = float(rate_tmp[2].split(' ')[1][:-1])

                for k in i.find_all('a', {'href' : re.compile(r'\d{5}')}):
                    ID = k['href'].split('/')[2]

                if ID == 0:
                    

        if type(i) == element.Tag:
            Flag = False
            for j in i.find_all('a', {'href' : re.compile(r'\d{6}')}):
                sub_url = j['href']
                card_ID = sub_url.split('_')[1]
                rarity = get_card_info(card_ID)['rarity']
            
                add_date = get_added_date("https://starlight.kirara.ca"+sub_url, card_ID).date()

                if rarity == 'SSR':
                    if datetime.date.today() - datetime.timedelta(days = 14) < add_date:
                        ssr_pickup_list_1.append(card_ID)
                    else:
                        ssr_pickup_list_0.append(card_ID)
                elif rarity == 'SR':
                    sr_pickup_list.append(card_ID)

            for k in i.find_all('a', {'href' : re.compile(r'\d{5}')}):
                ID = k['href'].split('/')[2]
            gacha_list.append({'ID' : ID, 'time' : float(time), 'sr_pickup_list_0' : sr_pickup_list, 'ssr_pickup_list_0' : ssr_pickup_list_0, 'ssr_pickup_list_1' : ssr_pickup_list_1, 'rate': rate})
        elif type(i) == element.Tag and i['class'][0] == 'header' and i.select('.left')[0].text == 'Limited availability!':
            time = i.select('.counter')[0]['data-count-to']

            if len(i.select('small')) != 0:
                rate_tmp = i.select('small')[0].text
                rate_tmp = rate_tmp.split('/')
                rate_tmp[0] = rate_tmp[0].split(':')[1]
                rate_tmp = list(map(lambda x : x.strip(), rate_tmp))

                rate['R'] = float(rate_tmp[0].split(' ')[1][:-1])
                rate['SR'] = float(rate_tmp[1].split(' ')[1][:-1])
                rate['SSR'] = float(rate_tmp[2].split(' ')[1][:-1])

            Flag = True    
    return gacha_list

In [2]:
res = requests.get("https://starlight.kirara.ca/")
html = res.text
soup = BeautifulSoup(html, 'html.parser')